<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [10]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [11]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
with open('../../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164445
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "finland", "city": "helsinki ", "street": "kallio-2", "aqicn_url": "https://api.waqi.info/feed/@5717", "latitude": 60.17, "longitude": 24.94}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [20]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [25]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df
query = air_quality_fg.select_all()
air_quality_df = query.read()
air_quality_df['date'] = pd.to_datetime(air_quality_df['date']).dt.tz_localize(None)
aq_today_df['date'] = pd.to_datetime(aq_today_df['date']).dt.tz_localize(None)
combined_df = pd.concat([air_quality_df, aq_today_df]).sort_values('date')

aq_today_df['pm25_lag_1'] = combined_df['pm25'].shift(1).iloc[-1]
aq_today_df['pm25_lag_2'] = combined_df['pm25'].shift(2).iloc[-1]
aq_today_df['pm25_lag_3'] = combined_df['pm25'].shift(3).iloc[-1]

aq_today_df['pm25_lag_1'] = aq_today_df['pm25_lag_1'].astype('float32')
aq_today_df['pm25_lag_2'] = aq_today_df['pm25_lag_2'].astype('float32')
aq_today_df['pm25_lag_3'] = aq_today_df['pm25_lag_3'].astype('float32')


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.21s) 


In [26]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pm25        1 non-null      float32       
 1   country     1 non-null      object        
 2   city        1 non-null      object        
 3   street      1 non-null      object        
 4   date        1 non-null      datetime64[ns]
 5   url         1 non-null      object        
 6   pm25_lag_1  1 non-null      float32       
 7   pm25_lag_2  1 non-null      float32       
 8   pm25_lag_3  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [27]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 60.25°N 25.0°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-20,1.35,1.9,37.936893,80.716522,helsinki
1,2024-11-21,2.90,0.1,33.114128,191.920715,helsinki
2,2024-11-22,-1.55,0.1,16.516901,249.590027,helsinki
3,2024-11-23,-1.80,0.0,4.198285,239.036301,helsinki
4,2024-11-24,-0.20,0.0,8.396570,239.036301,helsinki
5,2024-11-25,5.70,0.4,33.014175,205.167831,helsinki
6,2024-11-26,8.30,0.1,26.089443,230.599365,helsinki
7,2024-11-27,1.10,0.0,9.290511,305.537750,helsinki
8,2024-11-28,2.50,0.0,16.418526,254.744827,helsinki
9,2024-11-29,1.60,0.0,15.496736,267.337036,helsinki


In [28]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [29]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-20 02:09:11,286 INFO: 	4 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164445/fs/1155148/fg/1357149


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1164445/jobs/named/air_quality_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fadc1b59280>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 682000
         }
       },
       "result": {
         "observed_value": 39.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-20T01:09:11.000285Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
        

In [30]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-20 02:09:28,307 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164445/fs/1155148/fg/1350031


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1164445/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fadbf525760>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 673848
         }
       },
       "result": {
         "observed_value": 4.198285102844238,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-20T01:09:28.000307Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
